In [92]:
from gurobipy import *
import numpy as np

# Create model
model = Model("PGM-MILP")

# Create variables
x1 = model.addVar(lb = -1*GRB.INFINITY, ub= GRB.INFINITY, name = "x1") # Represents min(x, upper)
x2 = model.addVar(lb= -1*GRB.INFINITY, ub=GRB.INFINITY, name = "x2") # Represents max(x, lower)
x = model.addVar(lb = -1000, ub = 1000, name = "x") # Variable to tie x1 and x2 together 
b1 = model.addVar(vtype=GRB.BINARY, name = "b1") # Binary variable for min big M constraints
b2 = model.addVar(vtype=GRB.BINARY, name = "b2") # Binary variable for max big M constraints

#Set coefficient e -> Monte carlo'd over n trials 
n = 10000 # Number of trials 
e = 0 # Coefficient for x2 -> e ~ N(0,1)
i = 0
obj = LinExpr()
while i < n:
    norm = np.random.normal(0.0, 1.0)
    obj = obj + x1 + norm*x2
    e = e + norm # Used to check the sample average for testing
    i = i + 1
e = e/n
print("Sample avergae of e is: " + str(e) + "\n")

#Set objective 
model.setObjective((1/n)*obj, GRB.MAXIMIZE)

#Set constraints
M = 10000
upper = 0.0 # min(x, upper)
lower = 0.0 # max(x, lower)

# There are two versions of the constraints (one commented and the other not) - they should both be correct implementations 
'''
# Using Gurobi's built in min and max constraints 
minm = list()
minm.append(x)
minm.append(upper)
maxm = list()
maxm.append(x)
maxm.append(lower)
model.addGenConstrMin(x1, minm) # Asserts constraint that x1 is equal to the minimum of the list minm
model.addGenConstrMax(x2, maxm) # Asserts constraint that x2 is equal to the maximum of the list maxm
'''

# Using big M constraints to dictate the value of x1 and x2 depending on x

# Constraints for min(x, upper) -> b1 = 1 indicates that x >= upper, meaning x1 = upper. b1 = 0 indicates x <= upper and x1 = x
model.addConstr(x >= upper - M*(1-b1))
model.addConstr(x <= upper + M*b1)
model.addConstr(x1 >= upper - M*(1-b1))
model.addConstr(x1 <= upper + M*(1-b1))
model.addConstr(x1 >= x - M*b1)
model.addConstr(x1 <= x + M*b1)

# Constraints for max(x, lower) -> b2 = 1 indicates that x <= lower, meaning x2 = lower. b2 = 0 indicates x >= lower and x2 = x
model.addConstr(x <= lower + M*(1-b2))
model.addConstr(x >= lower - M*b2)
model.addConstr(x2 >= lower - M*(1-b2))
model.addConstr(x2 <= lower + M*(1-b2))
model.addConstr(x2 >= x - M*b2)
model.addConstr(x2 <= x + M*b2)

# Optimize model
model.optimize()

# Print Results 
for v in model.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % model.objVal)
                   


Sample avergae of e is: 0.0036198215824466105

Optimize a model with 12 rows, 5 columns and 28 nonzeros
Variable types: 3 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e-03, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+04, 1e+04]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 5 columns, 18 nonzeros
Variable types: 3 continuous, 2 integer (2 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 3.619822e+00, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.6198216    3.61982  0.00%     -    0s

Explored 0 nodes (4 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 3.61982 -0 
Pool objective bound 3.61982

Optimal solution found (t